# Finding best policy by brute force
On this example we will do the simplest way of finding a policy for the cart pole problem, by brute force. This problem is simple because the state space and action space is small.

#### State Space (1x4 vector)

Num | Observation | Min | Max
---|---|---|---
0 | Cart Position | -2.4 | 2.4
1 | Cart Velocity | -Inf | Inf
2 | Pole Angle | ~ -41.8&deg; | ~ 41.8&deg;
3 | Pole Velocity At Tip | -Inf | Inf

#### Action space (1x2 vector)

Num | Action
--- | ---
0 | Push cart to the left
1 | Push cart to the right

#### Search Space
On this problem the search space is 8 because the action space has 2 elements and the state space 4 elements:
$$len(\text{S}_{\text{1x4}}) . len(\text{A}_\text{1x2})$$

#### Episode Termination
* Pole Angle is more than ±12°
* Cart Position is more than ±2.4 (center of the cart reaches the edge of the display)
* Episode length is greater than 200

### References
* https://github.com/openai/gym/wiki/CartPole-v0
* https://medium.com/@m.alzantot/deep-reinforcement-learning-demystified-episode-0-2198c05a6124
* https://ray.readthedocs.io/en/latest/index.html
* https://bair.berkeley.edu/blog/2018/01/09/ray/

### Load Libraries and paramters

In [1]:
import time
import gym
import numpy as np

# Only log errors
gym.logger.set_level(40)

# Policies to generate
n_policy = 50000

### Policy
On this case our parametrized policy will receive our state $S_\text{1x4}$ and do a dot product with it's internal set of parameters $\theta_{1x4}$

In [2]:
# Generate random 1x4 vector for the policy parameters
def gen_random_policy_params():
    return (np.random.uniform(-1,1, size=4).astype(np.float32))

# Evaluate a policy given it's parameters and some state
def policy(policy_params, state):
    # It's basically a linear model
    if np.dot(policy_params, state) > 0:
        return 1
    else:
        return 0

### Generate a list of possilbe policies

In [3]:
policy_params_list = [gen_random_policy_params() for _ in range(n_policy)]
print('Generated list of %d policies parameters' % len(policy_params_list))

Generated list of 50000 policies parameters


### Evaluate Policy
This function basically run a policy during 

In [4]:
def evaluate_policy_episode(policy_params, render=False):
    # Crete environment
    env = gym.make('CartPole-v0')
    state = env.reset()
    # Total rewards from eposide
    total_reward_episode = 0
    while True:
        if render:
            env.render()
        selected_action = policy(policy_params, state)
        state, reward, done, _ = env.step(selected_action)
        total_reward_episode += reward
        if done:
            break
    return total_reward_episode

### Search for Policies

In [5]:
%%time
# Evaluate the policies on in the environment
scores_list = [evaluate_policy_episode(p) for p in policy_params_list]

/Users/leonardo_a/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


CPU times: user 25.3 s, sys: 56.4 s, total: 1min 21s
Wall time: 1min 21s


### Evaluate Best Policy

In [6]:
# Select the best policy from the score list.
best_score = max(scores_list)
print('Best policy score = %d' % best_score)

best_policy_params= policy_params_list[np.argmax(scores_list)]
print('Found best policy with params:', best_policy_params)

# Run best policy [0.32896566 0.56930596 0.8278743  0.43927363]
total_rewards_best_policy = evaluate_policy_episode(best_policy_params, render=True);
print('Total rewards on best policy:', total_rewards_best_policy)

if total_rewards_best_policy != best_score:
    print('Something went wrong')

Best policy score = 200
Found best policy with params: [-0.6190245   0.4777126   0.01823577  0.9735814 ]
Total rewards on best policy: 191.0
Something went wrong


### Distributed Brute Force
One of the advantages of the brute force method is that we can easily make things in parallel. For this example we will use the Ray framework to evalulate the list of policies in parallel.

```python
import time
import ray
ray.init()

def f1():
    time.sleep(1)

@ray.remote
def f2():
    time.sleep(1)

# The following takes ten seconds.
[f1() for _ in range(10)]

# The following takes one second (assuming the system has at least ten CPUs).
ray.get([f2.remote() for _ in range(10)])
```

### Import and instantiate the distributed computing server

In [7]:
import ray
ray.init()

2019-02-23 16:14:26,375	WARNING worker.py:1354 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-02-23 16:14:26,377	INFO node.py:278 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-02-23_16-14-26_45532/logs.
2019-02-23 16:14:26,486	INFO services.py:396 -- Waiting for redis server at 127.0.0.1:28055 to respond...
2019-02-23 16:14:26,607	INFO services.py:396 -- Waiting for redis server at 127.0.0.1:63329 to respond...
2019-02-23 16:14:26,613	INFO services.py:798 -- Starting Redis shard with 10.0 GB max memory.
2019-02-23 16:14:26,640	INFO services.py:1360 -- Starting the Plasma object store with 6.871947672999999 GB memory using /tmp.



View the web UI at http://localhost:8889/notebooks/ray_ui.ipynb?token=f44241cbae037a8d53c5ccd8699478c249f4d3973ab7855e



{'node_ip_address': None,
 'redis_address': '192.168.120.200:28055',
 'object_store_address': '/tmp/ray/session_2019-02-23_16-14-26_45532/sockets/plasma_store',
 'webui_url': 'http://localhost:8889/notebooks/ray_ui.ipynb?token=f44241cbae037a8d53c5ccd8699478c249f4d3973ab7855e',
 'raylet_socket_name': '/tmp/ray/session_2019-02-23_16-14-26_45532/sockets/raylet'}

In [8]:
@ray.remote
def evaluate_policy_episode_par(policy_params):
    # Crete environment
    env = gym.make('CartPole-v0')
    state = env.reset()
    # Total rewards from eposide
    total_reward_episode = 0
    while True:
        selected_action = policy(policy_params, state)
        state, reward, done, _ = env.step(selected_action)
        total_reward_episode += reward
        if done:
            break
    return total_reward_episode

### Evaluate Policies in the cluster

In [9]:
%%time
scores_list_par = ray.get([evaluate_policy_episode_par.remote(p) for p in policy_params_list])

CPU times: user 7.65 s, sys: 900 ms, total: 8.55 s
Wall time: 1min 38s


### Evaluate Best Policy

In [12]:
# Select the best policy from the score list.
best_score = max(scores_list)
print('Best policy score = %d' % best_score)

best_policy_params= policy_params_list[np.argmax(scores_list_par)]
print('Found best policy with params:', best_policy_params)

# Run best policy [0.32896566 0.56930596 0.8278743  0.43927363]
total_rewards_best_policy = evaluate_policy_episode(best_policy_params, render=True);
print('Total rewards on best policy:', total_rewards_best_policy)

if total_rewards_best_policy != best_score:
    print('Something went wrong')

Best policy score = 200
Found best policy with params: [-0.38530764  0.32140714  0.53714424  0.91360974]
Total rewards on best policy: 200.0
